# Preprocessing, building a Pandas dataframe and saving it as a  .csv file

In [6]:
import re
import sys
import glob
import string
from pprint import pprint
from collections import Counter, OrderedDict

import spacy
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')
from sklearn.metrics import accuracy_score


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

folder_path = "/home/niki/Documents/BE_Project/my_EmailRecommmendation/Scraping/mini_deb/*"
file_name = "/home/niki/Documents/BE_Project/my_EmailRecommendation/dataframe3.csv"
sys.path.insert(0, '/home/niki/Documents/BE_Project/EmailRecommendation/Preprocessing')
MODEL_PATH = "/home/niki/Documents/BE_Project/EmailRecommendation/Preprocessing/infersent1.pkl"
W2V_PATH = '/home/niki/Documents/BE_Project/my_EmailRecommmendation/glove.6B/glove.6B.300d.txt'


import preprocessing
import read_file
import datetime

def extract_debian(text):
    text = text.split('\n\n\n')
    header = text[0].split('\n')
    body = text[1]
    sender = header[2].split(':')[1].split('<')[0]
#     print('Sender',sender)
#     print('Body \n',body)
    return sender,body

def clean_debian(temp):
    temp = re.sub('\n+','\n',temp)
    temp = re.sub('\n',' ',temp)
    temp = re.sub('\t',' ',temp)
    temp = re.sub(' +',' ',temp)
    return temp

def deb_lemmatize(doc):        
    doc = nlp(doc)
    article, skl_texts = '',''
    for w in doc:
        if w.text != '\n' and not w.is_stop and not w.is_punct and not w.like_num:
            article += " " + w.lemma_
        if w.text == '\n':                
            skl_texts += " " + article
            article = ''       
    return skl_texts

def deb_toppostremoval(temp):
    strings = temp.splitlines()
    temp = ''
    for st in strings:
        st = st.strip()
        if len(st)>0:
            if st[0]=='>':
                continue
            else:
                temp += '\n' + st
    return temp



In [7]:
from random import randint
import numpy as np
import torch
from models import InferSent
model_version = 1
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
infermodel = InferSent(params_model)
infermodel.load_state_dict(torch.load(MODEL_PATH))
use_cuda = False
infermodel = infermodel.cuda() if use_cuda else infermodel
#replace with glove.840B.300d.txt
infermodel.set_w2v_path(W2V_PATH)
infermodel.build_vocab_k_words(K=100000)

Vocab size : 100000


In [8]:
df = pd.DataFrame(columns=['body','replier', 'thread_no','embeddings'])
folder = glob.glob(folder_path)
th_no = 0
obj = preprocessing.preprocess()
cnt = 0
count_file = 0
thread_list=[]
try:
    for fol in tqdm_notebook(folder):
        files = glob.glob(fol+'/*.txt')
        threads = []
        for file in files:
            ob = read_file.file_content(file)
            ob.read_file_content()
            threads.append(ob.mail)
            count_file += 1
        sorted_threads = sorted(threads, key=lambda ke: datetime.datetime.strptime(ke['Date'],'%a, %d %b %Y %H:%M:%S %z'))
        thread_list.append(sorted_threads)
except:
    print(fol)
print(len(thread_list))

HBox(children=(IntProgress(value=0, max=107), HTML(value='')))


107


In [ ]:
for thr in thread_list:
    flag = 0
    t = ''
    for mail in thr:
        temp = ''
        count_file += 1
        sender = mail['From']
        temp   = mail['content']
        temp = deb_toppostremoval(temp)
        temp = deb_lemmatize(temp)
        temp = clean_debian(temp)
        if temp == '':
            cnt += 1
            print('NULL')
            continue
        temp = obj.replace_tokens(temp)
        if flag==0:
            t = temp
            flag = 1
            continue
        t=t.strip()
        print(str(t))
        print('---------------------------------------')
        #calculate sentence embedding for body and average it into 4096 sized vector
        if t!='' :
            embedding =infermodel.encode( str(t), bsize=1, tokenize=False, verbose=True)
            sent_vec =[]
            numw = 0
            for w in embedding:
                try:
                    if numw == 0:
                        sent_vec = w
                    else:
                        sent_vec = np.add(sent_vec, w)
                    numw+=1
                except:
                    pass
            v = np.asarray(sent_vec) / numw
            print(v.shape)
            print(v)
            v=np.transpose(v)
            print(v.shape)
            print(v)
        v=np.zeros(4096)
        df = df.append({'body': str(t),'replier':sender, 'thread_no':th_no,'embeddings':v}, ignore_index=True)
        t = t + temp
    th_no += 1

empty.close()
print(cnt)
print(count_file)
print(len(df['body']))
print(len(df['thread_no'].unique()))
print(len(df['replier'].unique()))
print(len(df['embeddings'][0]))
rep_to_index = {}
index = 0
for rep in df['replier']:
    if rep in rep_to_index:
        continue
    else:
        rep_to_index[rep] = index
        index += 1
pprint(len(rep_to_index))
for rep in df['replier']:
    df.loc[df['replier']==rep,'replier'] = rep_to_index[rep]

# Aggregate according to date / make separate thread list |> P flag
# Split test_train_split 
print(df.head)
df.to_csv(file_name)
unique_users = len(df.replier.unique())

On Thu Mar  PM Rebecca N. Palmer   write I think lot appstream install KDE GNOME install plasma discover gnome software default But beignet opencli icd have appstream metadata invalid
---------------------------------------
Nb words kept : 183/518 (35.3%)
Speed : 69.2 sentences/s (cpu mode, bsize=1)
(4096,)
[ 8.4855227e-04 -9.4013400e-03  2.4437355e-02 ... -2.7969321e-02
  2.7301970e-05 -4.8896428e-03]
(4096,)
[ 8.4855227e-04 -9.4013400e-03  2.4437355e-02 ... -2.7969321e-02
  2.7301970e-05 -4.8896428e-03]
On Thu Mar  PM Rebecca N. Palmer   write I think lot appstream install KDE GNOME install plasma discover gnome software default But beignet opencli icd have appstream metadata invalid On   Jeremy Bicha write Do thing display AppStream package relate hardware default This part metadata automatically extract LGPL code have warn appstreamcli validate people I discuss time think OK Upstream grant permission metadata MIT late freeze The homepage field metadata set
-------------------------

Speed : 91.7 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00178447 -0.00549774  0.03256304 ... -0.02598251  0.00320778
 -0.00105744]
(4096,)
[ 0.00178447 -0.00549774  0.03256304 ... -0.02598251  0.00320778
 -0.00105744]
Dearend I need advice development Debian I use test repo My desktop environment Gnome I contribute Gnome have project For big challenge work upstream library Basically question lay plane maintain upstream library I use project For I not build library dependency available debian repo I know jhbuild library e.g. libgdamm available module I check Flatpak not figure use library development I want ask advice share experience matt helpful Thanks Pavlo Solntsev Sent Evolution GNU Debian On Thu Apr  AM Pavlo Solntsev write Your option Manually build libgdamm etc install ~ opt usr local Talk jhbuild folk get libgdamm etc Talk libgdamm etc folk add GNOME runtime Package libgdamm etc .deb form Debian experimental Dear Paul Thank quick response work easily I need rebuild libgda glib

Speed : 83.5 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00262757 -0.00336174  0.03098311 ... -0.02816987  0.00234667
 -0.00179953]
(4096,)
[ 0.00262757 -0.00336174  0.03098311 ... -0.02816987  0.00234667
 -0.00179953]
On Thu Aug PM  Andreas Henriksson write Following previous mail debian devel announce An initial set bug report file look sfdisk user codesearch.debian.net promise See
---------------------------------------
Nb words kept : 167/476 (35.1%)
Speed : 76.3 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00469041 -0.00066562  0.03325256 ... -0.02503725  0.00395855
  0.00039533]
(4096,)
[ 0.00469041 -0.00066562  0.03325256 ... -0.02503725  0.00395855
  0.00039533]
On Thu Aug PM  Andreas Henriksson write Following previous mail debian devel announce An initial set bug report file look sfdisk user codesearch.debian.net promise See On Thu Aug  PM Andreas Henriksson   write Why set recommand It install default deinstalled
---------------------------------------
Nb words kept : 258/732

Speed : 87.1 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00362954 -0.00179768  0.0310603  ... -0.02840232  0.00250708
  0.00267104]
(4096,)
[ 0.00362954 -0.00179768  0.0310603  ... -0.02840232  0.00250708
  0.00267104]
On Thu Apr AM  Russ Allbery write I agree need fix This fact open interpretation I subscribe edge case argument In dir I nearly Git repository Debian Med team I  find name debian type have  grep v \.git  wc l  find name orig source type f  wc l I check single recent knowledge obtain source Git repository uscan droped But roughly percent package release package finish preparation admittedly percentage orig source script high PRON be happy downgrade policy Developer have Reference I love precise word find policy long settle clarify word have OK Kind regard Andreas Hi Sean Whitton write PRON be bite confuse not specify pretty precisely I favor add target description line If provide orig source targetisfy  interface. If provide orig source target debian watch file use archiv

Speed : 88.0 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00309506 -0.00498623  0.02995902 ... -0.02685394  0.00233386
  0.00165035]
(4096,)
[ 0.00309506 -0.00498623  0.02995902 ... -0.02685394  0.00233386
  0.00165035]
On Thu Nov PM  Holger Levsen write I not PRON be glad mean way go mind future reference people try rhetorical PRON will pretend not discussion forward constructive shut Me PRON have partly I suggestion Code Conduct To thief steal anti depressant I hope be happy
---------------------------------------
Nb words kept : 261/737 (35.4%)
Speed : 88.3 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00214926 -0.00309122  0.03019569 ... -0.02817906  0.00357418
  0.00025645]
(4096,)
[ 0.00214926 -0.00309122  0.03019569 ... -0.02817906  0.00357418
  0.00025645]
On Thu Nov PM  Holger Levsen write I not PRON be glad mean way go mind future reference people try rhetorical PRON will pretend not discussion forward constructive shut Me PRON have partly I suggestion Code Conduct To thief stea

...

Speed : 58.1 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00585227 -0.00806382  0.03042969 ... -0.0270344   0.0003325
  0.00215222]
(4096,)
[ 0.00585227 -0.00806382  0.03042969 ... -0.0270344   0.0003325
  0.00215222]
On Fri Dec AM  Joerg Jaspert write As today bug maintain package get fix aptitude lintian packagekit reportbug synaptic vim neovim zsh libconfig model dpkg perl gambas All appear track test freeze dln dpkg Josh Triplett write PRON have
---------------------------------------
Nb words kept : 235/666 (35.3%)
Speed : 73.1 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00571176 -0.00773548  0.02986997 ... -0.0278941   0.00049552
  0.00068226]
(4096,)
[ 0.00571176 -0.00773548  0.02986997 ... -0.0278941   0.00049552
  0.00068226]
On Fri Dec AM  Joerg Jaspert write As today bug maintain package get fix aptitude lintian packagekit reportbug synaptic vim neovim zsh libconfig model dpkg perl gambas All appear track test freeze dln dpkg Josh Triplett write PRON have On Mon Jan PM  Ansga

Speed : 70.0 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00424061 -0.00523248  0.03421255 ... -0.02753188  0.00436953
 -0.00180534]
(4096,)
[ 0.00424061 -0.00523248  0.03421255 ... -0.02753188  0.00436953
 -0.00180534]
Added runit maining list thread  Bastian Blank   Hello debian Above REJECT ftp master suggest discuss issue     Background    I maintain runit init supervision system ^ Debian ^ Runit init system compatible extend SysVInit able use traditional init.d script But use supervision possiblities service define format native runit Here typical runscript look like  tree etc sv wicd daemon etc sv wicd daemon ├ ─ ─ log │    ├ ─ ─ run │    └ ─ ─ supervise  var lib runit log supervise wicd daemon ├ ─ ─ run └ ─ ─ supervise  var lib runit supervise wicd daemon provide  run package  dpkg L wicd daemon run etc sv wicd daemon log etc sv wicd daemon log run etc sv wicd daemon run var log runit wicd daemon Essential  run script program work foreground In particular case look like follow bi

Speed : 78.8 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00503513 -0.00790874  0.02675281 ... -0.02792985 -0.00056448
 -0.00450477]
(4096,)
[ 0.00503513 -0.00790874  0.02675281 ... -0.02792985 -0.00056448
 -0.00450477]
NULL
Hello The BuildProfileSpec wiki define  nocheck  profile purpose ignore Builds set profile add  nocheck   DEB_BUILD_OPTIONS    I understand say debhelper step dh_auto_test override_dh_auto_test automatically skip profile define Am I mistake PRON be ask PRON have see follow pattern couple package   ifeq  findstring nocheck  DEB_BUILD_OPTIONS override_dh_auto_test command command override_dh_auto_test endif   Thanks regard Afif
---------------------------------------
Nb words kept : 429/1223 (35.1%)
Speed : 75.3 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00547249 -0.00379907  0.02505635 ... -0.02750573  0.00059403
  0.00082656]
(4096,)
[ 0.00547249 -0.00379907  0.02505635 ... -0.02750573  0.00059403
  0.00082656]
Hello The BuildProfileSpec wiki define  nocheck  profi

Speed : 79.5 sentences/s (cpu mode, bsize=1)
(4096,)
[ 1.4295705e-03  2.1785692e-05  3.2021191e-02 ... -2.5780128e-02
  4.8036617e-03  2.8297373e-03]
(4096,)
[ 1.4295705e-03  2.1785692e-05  3.2021191e-02 ... -2.5780128e-02
  4.8036617e-03  2.8297373e-03]
Hi First clear I think decomissioning Alioth good thing give current state Thanks One concern will need mass switch git repo URLs nth time The nice thing current setup have decouple Alioth URL wise I think nice preserve And relate concern current candidate gitlab suffer flat namespace disease github user orgs group team conflate site user centric instead project team center For like gitlab.com github.com have okay passable annoy like Debian user registration independent forge problem collision wait happen consider group team freely When like package maintain team team compose member have nice way group relate package PRON would like note gitlab gitea wrong pagure example get right Thanks
---------------------------------------
Nb words

Speed : 77.9 sentences/s (cpu mode, bsize=1)
(4096,)
[ 0.00429476 -0.00793628  0.03583552 ... -0.02730151  0.00383983
  0.00042321]
(4096,)
[ 0.00429476 -0.00793628  0.03583552 ... -0.02730151  0.00383983
  0.00042321]
rhkramer Intentionally cross post Aside For debian user list thread come debian backports list frustration probably express debian user list debian developer list assume list I subscribe But name use name get frustrate I suspect I The number version Toy Story name test stable old stable old old stable frustrate Tangentially need pick drop baton update picture
---------------------------------------
Nb words kept : 361/1027 (35.2%)


# Indexing of words in vocab

In [ ]:
words = Counter()
for sent in df.body.values:
    words.update(w.text.lower() for w in nlp(sent))
# print(words)

words = sorted(words, key=words.get, reverse=True)
print(words)
words = ['_PAD','_UNK'] + words

word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}
def indexer(s): return [word2idx[w.text.lower()] for w in nlp(s)]

# Dataset Loading

In [106]:
# embedding |> flag
class VectorizeData(Dataset):
    def __init__(self, df_path, maxlen=4096):
        self.df = pd.read_csv(df_path, error_bad_lines=False)
        print(self.df.embeddings)
        self.df['body'] = self.df.body.apply(lambda x: x.strip())
        print('Indexing...')
        self.df['bodyidx'] = self.df.body.apply(indexer)
        print('Calculating lengths')
        self.df['lengths'] = 4096
        self.maxlen = 4096
        print(self.maxlen)
        print('Padding')
        self.df['bodypadded'] = self.df.bodyidx.apply(self.pad_data)
     
        
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        X = self.df.bodypadded[idx]
        lens = self.df.lengths[idx]
        y = self.df.replier[idx]
        e=self.df.embeddings[idx]
        return X,y,lens,e
    
    def pad_data(self, s):
        padded = np.zeros((self.maxlen,), dtype=np.int64)
        if len(s) > self.maxlen: padded[:] = s[:self.maxlen]
        else: padded[:len(s)] = s
        return padded

In [107]:
ds = VectorizeData(file_name)

0                           []
1                           []
2                           []
3                           []
4                           []
5                           []
6                           []
7                           []
8                           []
9                           []
10                          []
11                          []
12                          []
13                          []
14                          []
15                          []
16                          []
17                          []
18                          []
19                          []
20                          []
21                          []
22                          []
23                          []
24                          []
25                          []
26                          []
27                          []
28                          []
29                          []
                ...           
437    [0. 0. 0. ... 0. 0. 0.]
438    [

In [108]:
input_size = 4096
# input_size = ds.maxlen
hidden_size = 30
num_classes = unique_users
num_epochs = 1
batch_size = 1
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [109]:
# concatenate user vector
# embeddings |>
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
#         x = torch.FloatTensor(x)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [110]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [112]:
train_dl= DataLoader(ds, batch_size=1)
num_batch = len(train_dl)
for epoch in range(num_epochs):
    y_true_train = list()
    y_pred_train = list()
    total_loss_train = 0
    t = tqdm_notebook(iter(train_dl), leave=False, total=num_batch)
    for X,y, lengths,e in t:
    #     X = X.transpose(0,1)
        X = Variable(X.cpu())
        y = Variable(y.cpu())
        lengths = lengths.numpy()
        print(X,y,lengths,e)    
        opt.zero_grad()
        X = X.float()
        pred = model(e)```````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````````
        # F.nll_loss can be replaced with criterion
        loss = F.nll_loss(pred, y)
        loss.backward()
        opt.step()

        t.set_postfix(loss=loss.data[0])
        pred_idx = torch.max(pred, dim=1)[1]

        y_true_train += list(y.cpu().data.numpy())
        y_pred_train += list(pred_idx.cpu().data.numpy())
        total_loss_train += loss.data[0]

    train_acc = accuracy_score(y_true_train, y_pred_train)
    train_loss = total_loss_train/len(train_dl)
    print(f' Epoch {epoch}: Train loss: {train_loss} acc: {train_acc}')
#Accuracy
# hyperparameter tuning
#Testing
# architecture testing

tensor([[ 11, 198, 167,  ...,   0,   0,   0]]) tensor([0]) [4096] ('[]',)


AttributeError: 'tuple' object has no attribute 'dim'